# **Querying Data with SQLite (Part 1)**

In this Jupyter Notebook, we demonstrate basic SQL queries using an SQLite database. We will explore the tables to view the database structures, use basic SQL queries for selecting and filtering data, aggregate and group the data, as well as implement sorting, limiting, and offsetting in queries. To accomplish this, we will use the SQLite database `SQL_basic_examples.db`.

***

## **1. Setup**

### **1.1 ~ Import Libraries**

In [1]:
# Import necessary libraries
import sqlalchemy as db
import pandas as pd

### **1.2 ~ Connect to the SQLite database**

In [2]:
# Create connection to the SQLite database
engine = db.create_engine('sqlite:///../data/SQL_basic_examples.db')
conn = engine.connect()

In [3]:
# View all tables in the database
query = """
SELECT *
FROM sqlite_master
WHERE type='table';
"""

df = pd.read_sql(query, conn)
display(df)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,7,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Dragon,Dragon,2,CREATE TABLE Dragon (\n name TEXT PRIMARY K...
2,table,Dish,Dish,4,CREATE TABLE Dish (\n name TEXT PRIMARY KEY...
3,table,Scene,Scene,6,CREATE TABLE Scene (\n id INTEGER PRIMARY K...


***

## **2. Querying Data from the 'Dragon' Table**

### **2.1 ~ Selecting All Columns**

In [4]:
# Query to select all columns from the 'Dragon' table
query = """
SELECT *
FROM Dragon;
"""

df = pd.read_sql(query, conn)
display(df)

,name,year,cute
0,hiccup,2010,10
1,drogon,2011,-100
2,dragon 2,2019,0


### **2.2 ~ Selecting Specific Columns**

In [5]:
# Select specific columns from the 'Dragon' table
query = """
SELECT name, year
FROM Dragon;
"""

df = pd.read_sql(query, conn)
display(df)

,name,year
0,hiccup,2010
1,drogon,2011
2,dragon 2,2019


### **2.3 ~ Renaming Columns**

In [6]:
# Rename columns using the AS keyword
query = """
SELECT name AS dragon_name, year AS birth_year
FROM Dragon;
"""

df = pd.read_sql(query, conn)
display(df)

,dragon_name,birth_year
0,hiccup,2010
1,drogon,2011
2,dragon 2,2019


***

## **3. Filtering Data with the WHERE Clause**

### **3.1 ~ Selecting Cute Dragons**

In [7]:
# Conditional selection using the WHERE clause
query = """
SELECT *
FROM Dragon
WHERE cute > 0;
"""

df = pd.read_sql(query, conn)
display(df)

,name,year,cute
0,hiccup,2010,10


### **3.2 ~ Selecting Dragons by Cuteness or Birth Year**

In [8]:
# Select dragons that are either cute or born after 2013
query = """
SELECT *
FROM Dragon
WHERE cute > 0 OR year >= 2013;
"""

df = pd.read_sql(query, conn)
display(df)

,name,year,cute
0,hiccup,2010,10
1,dragon 2,2019,0


***

## **4. Sorting and Limiting Query Results**

### **4.1 ~ Sorting Dragons by Cuteness**

In [9]:
# Order the 'Dragon' table by the 'cute' column in descending order
query = """
SELECT *
FROM Dragon
ORDER BY cute DESC;
"""

df = pd.read_sql(query, conn)
display(df)

,name,year,cute
0,hiccup,2010,10
1,dragon 2,2019,0
2,drogon,2011,-100


### **4.2 ~ Limiting the Number of Rows**

In [10]:
# Limit the number of selected rows to 2
query = """
SELECT *
FROM Dragon
ORDER BY cute DESC
LIMIT 2;
"""

df = pd.read_sql(query, conn)
display(df)

,name,year,cute
0,hiccup,2010,10
1,dragon 2,2019,0


### **4.3 ~ Using OFFSET to Start from a Specific Row**

In [11]:
# Limit the number of rows to 2, but start from the second row (offset 1)
query = """
SELECT *
FROM Dragon
ORDER BY cute DESC
LIMIT 2
OFFSET 1;
"""

df = pd.read_sql(query, conn)
display(df)

,name,year,cute
0,dragon 2,2019,0
1,drogon,2011,-100


***

## **5. Querying Data from the 'Dish' Table**

### **5.1 ~ Selecting All Rows from 'Dish'**

In [12]:
# Select all rows from the 'Dish' table
query = """
SELECT *
FROM Dish;
"""

df = pd.read_sql(query, conn)
display(df)

,name,type,cost
0,ravioli,entree,10
1,pork bun,entree,7
2,taco,entree,7
3,edamame,appetizer,4
4,fries,appetizer,4
5,potsticker,appetizer,4
6,ice cream,dessert,5


### **5.2 ~ Grouping Rows by 'Type'**

In [13]:
# Group rows by 'type' in the 'Dish' table
query = """
SELECT type, COUNT(*) AS count
FROM Dish
GROUP BY type;
"""

df = pd.read_sql(query, conn)
display(df)

,type,count
0,appetizer,3
1,dessert,1
2,entree,3


### **5.3 ~ Aggregating with Minimum Cost per Group**

In [14]:
# Group by 'type' and get the minimum cost per group
query = """
SELECT type, MIN(cost) AS min_cost FROM Dish
GROUP BY type;
"""

df = pd.read_sql(query, conn)
display(df)

,type,min_cost
0,appetizer,4
1,dessert,5
2,entree,7


***

## **6. Using Multiple Aggregations**

### **6.1 ~ Multiple Aggregation Functions**

In [15]:
# Define multiple aggregation functions: min(cost) and max(name)
query = """
SELECT type, MIN(cost) AS min_cost, MAX(name) AS max_name
FROM Dish
GROUP BY type;
"""

df = pd.read_sql(query, conn)
display(df)

,type,min_cost,max_name
0,appetizer,4,potsticker
1,dessert,5,ice cream
2,entree,7,taco


### **6.2 ~ More Aggregations with Min, Avg, and Max Costs**

In [16]:
# Multiple aggregation functions: min, avg, max costs, and max name
query = """
SELECT type, MIN(cost) AS min_cost, AVG(cost) AS avg_cost, MAX(cost) AS max_cost, MAX(name) AS max_name
FROM Dish
GROUP BY type;
"""

df = pd.read_sql(query, conn)
display(df)

,type,min_cost,avg_cost,max_cost,max_name
0,appetizer,4,4.0,4,potsticker
1,dessert,5,5.0,5,ice cream
2,entree,7,8.0,10,taco


***

## **7. Grouping by Multiple Columns**

### **7.1 ~ Grouping by 'Type' and 'Cost'**

In [17]:
# Group by multiple columns (type and cost), and select max(name)
query = """
SELECT type, cost, MAX(name) AS max_name
FROM Dish
GROUP BY type, cost;
"""

df = pd.read_sql(query, conn)
display(df)

,type,cost,max_name
0,appetizer,4,potsticker
1,dessert,5,ice cream
2,entree,7,taco
3,entree,10,ravioli


### **7.2 ~ Adding a Count of Rows for Each Group**

In [18]:
# Add a third column that gives the number of rows for each group (count(*))
query = """
SELECT type, cost, MAX(name) AS max_name, COUNT(*) AS count
FROM Dish
GROUP BY type, cost;
"""

df = pd.read_sql(query, conn)
display(df)

,type,cost,max_name,count
0,appetizer,4,potsticker,3
1,dessert,5,ice cream,1
2,entree,7,taco,2
3,entree,10,ravioli,1


***

## **8. Filtering Grouped Data with HAVING**

### **8.1 ~ Using the HAVING Clause to Filter Groups**

In [19]:
# Only show types where the maximum cost is less than 8
query = """
SELECT type, COUNT(*) AS count
FROM Dish
GROUP BY type
HAVING MAX(cost) < 8;
"""

df = pd.read_sql(query, conn)
display(df)

,type,count
0,appetizer,3
1,dessert,1


***

## **9. Working with DISTINCT Values**

### **9.1 ~ Selecting Unique Types from 'Dish'**

In [20]:
# Select distinct types from the 'Dish' table
query = """
SELECT DISTINCT type
FROM Dish;
"""

df = pd.read_sql(query, conn)
display(df)

,type
0,entree
1,appetizer
2,dessert


### **9.2 ~ Using DISTINCT with Multiple Columns**

In [21]:
# Select distinct combinations of type and name
query = """
SELECT DISTINCT type, name
FROM Dish;
"""

df = pd.read_sql(query, conn)
display(df)

,type,name
0,entree,ravioli
1,entree,pork bun
2,entree,taco
3,appetizer,edamame
4,appetizer,fries
5,appetizer,potsticker
6,dessert,ice cream


### **9.3 ~ Using DISTINCT in Column Expressions**

In [22]:
# Use DISTINCT in column expressions to get the average of distinct costs per type
query = """
SELECT type, AVG(DISTINCT cost) AS avg_distinct_cost
FROM Dish
GROUP BY type;
"""

df = pd.read_sql(query, conn)
display(df)

,type,avg_distinct_cost
0,appetizer,4.0
1,dessert,5.0
2,entree,8.5
